In [3]:
# Exploration des bases de données enregistrées dans minio
# Create filesystem object
import os

import s3fs
import pandas as pd
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "radjerad"
FILE_KEY_S3 = "inpi/inpi_2019.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3
with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df = pd.read_csv(file_in, sep=",")

/tmp/ipykernel_468/745401279.py:13: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,22,23,24,25,27,28,31,32,34,35,36,38,39,40,41,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,64,65,66,67,68,69,70,72,73,74,75,76,77,78,79,80,81,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,100,101,102,103,105,106,107,108,109,110,111,112,113,114,115,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bpe = pd.read_csv(file_in, sep=",")


In [6]:
df.head()

,Unnamed: 0,Code Greffe,Nom_Greffe,Numero_Gestion,Siren,Type_Inscription,Date_Immatriculation,Date_1re_Immatriculation,Date_Radiation,Date_Transfert,...,Enseigne,Activité_Ambulante,Activité_Saisonnière,Activité_Non_Sédentaire,Date_Début_Activité,Activité,Origine_Fonds,Origine_Fonds_Info,Type_Exploitation,ID_Etablissement
0,0,101,Bourg-en-Bresse,2019B00890,305148884,S,2019-05-24,2019-05-24,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,101,Bourg-en-Bresse,1976B40029,308529668,P,1976-11-26,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,101,Bourg-en-Bresse,2002D00732,309729879,P,2002-10-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,101,Bourg-en-Bresse,2011B01046,317272243,P,2011-09-06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,101,Bourg-en-Bresse,1997B00138,319026720,P,1997-03-03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df["Siren"].nunique()
# 269 915

269915

In [9]:
df.shape # 5 888 180

(5888180, 119)

In [10]:
df["Date_Immatriculation"].head()

0    2019-05-24
1    1976-11-26
2    2002-10-30
3    2011-09-06
4    1997-03-03
Name: Date_Immatriculation, dtype: object

In [12]:
df["annee_Immatriculation"] = pd.to_datetime(df["Date_Immatriculation"]).dt.year

In [13]:
df["annee_Immatriculation"].value_counts() # ok donc pour le fichier stock 2019, on a le plus d'immatriculation en 2017

2017.0    42786
2018.0    23971
2019.0    13812
2016.0    13416
2015.0    11713
          ...  
1937.0        2
1922.0        1
1896.0        1
1918.0        1
1928.0        1
Name: annee_Immatriculation, Length: 104, dtype: int64

In [14]:
df.columns

Index(['Unnamed: 0', 'Code Greffe', 'Nom_Greffe', 'Numero_Gestion', 'Siren',
       'Type_Inscription', 'Date_Immatriculation', 'Date_1re_Immatriculation',
       'Date_Radiation', 'Date_Transfert',
       ...
       'Activité_Ambulante', 'Activité_Saisonnière', 'Activité_Non_Sédentaire',
       'Date_Début_Activité', 'Activité', 'Origine_Fonds',
       'Origine_Fonds_Info', 'Type_Exploitation', 'ID_Etablissement',
       'annee_Immatriculation'],
      dtype='object', length=120)

In [19]:
df.columns = [c.lower() for c in df.columns]

In [21]:
pd. set_option("display.max_columns", None)

In [25]:
df.columns = [c.replace("é", "e") for c in df.columns]

In [27]:
df.loc[~df["date_cessation_activite"].isna(), "date_cessation_activite"].head()

6     2018-10-31
8     2019-03-31
19    2019-04-01
25    2013-03-25
28    2007-11-21
Name: date_cessation_activite, dtype: object

In [28]:
df["denomination"].head()

,denomination,denomination
0,GROUPE SAB - FINANCIERE D'AZOLETTE,NaN
1,S.M.G. TURTSCHI,NaN
2,"SOCIETE CIVILE IMMOBILIERE BERJADANE, PAR ABRE...",NaN
3,PROMOTION IMMOBILIERE,NaN
4,ETABLISSEMENTS SABY GIRARDOT,NaN


In [29]:
df["siren"].isna().sum()

0

In [30]:
(df["siren"]=="").sum()

0

In [31]:
df["siren"].head()

0    305148884
1    308529668
2    309729879
3    317272243
4    319026720
Name: siren, dtype: int64

In [32]:
df["siren"].nunique() # 269 915

269915

In [37]:
df.loc[df["siren"] == "000000000",:].shape

(0, 120)

In [35]:
# personnes physiques
df.loc[~df["nom_patronymique"].isna(), "nom_patronymique"].head()

1453         CRIBEL
1454       TURTSCHI
1455       TURTSCHI
1456    TARTAGLIONE
1457    TARTAGLIONE
Name: nom_patronymique, dtype: object

In [36]:
df.loc[~df["nom_patronymique"].isna(), "nom_patronymique"].shape # 651 190

(651190,)

In [38]:
df.loc[~df["nom_usage"].isna(), "nom_usage"].shape # 114 577

(114577,)

In [39]:
df.loc[~df["prenoms"].isna(), "prenoms"].head()

1453    Georges
1454     Arnaud
1455      Cyril
1456      André
1457      André
Name: prenoms, dtype: object

In [40]:
df.loc[~df["date_naissance"].isna(), "date_naissance"].head()

1453    1989-08-31
1454    1985-06-11
1455    1977-01-14
1456    1945-03-17
1457    1945-03-17
Name: date_naissance, dtype: object

In [41]:
df.loc[~df["ville_naissance"].isna(), "ville_naissance"].head()

1453              71270 Mâcon (71)
1454             01034 Belley (01)
1455             01034 Belley (01)
1456    01053 Bourg-en-Bresse (01)
1457    01053 Bourg-en-Bresse (01)
Name: ville_naissance, dtype: object

In [43]:
(df["siren"]==0).value_counts()

False    5888089
True          91
Name: siren, dtype: int64

In [44]:
df["source"].value_counts()

6901_S2_20190506_12_actes.csv              174728
7501_S2_20190506_12_actes.csv              168471
9401_S3_20190506_12_actes.csv              154666
3102_S2_20190506_12_actes.csv               92834
6901_S2_20190506_13_comptes_annuels.csv     89147
                                            ...  
6901_S3_20190616_3_PP.csv                       1
2104_S3_20190616_3_PP.csv                       1
9201_S4_20190616_3_PP.csv                       1
7702_S4_20190616_3_PP.csv                       1
0203_S4_20190922_3_PP.csv                       1
Name: source, Length: 4638, dtype: int64

In [45]:
pm = df.loc[df["source"].str.contains('PM'),:]

In [46]:
pm.shape # 289 656

(289656, 120)

In [49]:
pp = df.loc[df["source"].str.contains('PP'),:]

In [50]:
pp.shape # 31 746

(31746, 120)

In [51]:
ets = df.loc[df["source"].str.contains('ets'),:]
ets.shape # 550 229

(550229, 120)

In [52]:
rep = df.loc[df["source"].str.contains('rep'),:]
rep.shape # 729 802

(729802, 120)

In [53]:
obs = df.loc[df["source"].str.contains('obs'),:]
obs.shape # 529 362

(529362, 120)

In [54]:
actes = df.loc[df["source"].str.contains('actes'),:]
actes.shape  # 2 579 430

(2579430, 120)

In [55]:
comptes_an = df.loc[df["source"].str.contains('comptes_annuels'),:]
comptes_an.shape # 1 177 955

(1177955, 120)

In [57]:
comptes_an.shape[0] + pm.shape[0] + pp.shape[0] + ets.shape[0] + actes.shape[0] + rep.shape[0] + obs.shape[0]

5888180

In [58]:
df.shape

(5888180, 120)